## # Introduction
<p><img src="https://i.imgur.com/kjWF1So.jpg" alt="Different characters on a computer screen"></p>
<p>According to a 2019 <a href="https://storage.googleapis.com/gweb-uniblog-publish-prod/documents/PasswordCheckup-HarrisPoll-InfographicFINAL.pdf">Google / Harris Poll</a>, 24% of Americans have used common passwords, like <code>abc123</code>, <code>Password</code>, and <code>Admin</code>. Even more concerning, 59% of Americans have incorporated personal information, such as their name or birthday, into their password. This makes it unsurprising that 4 in 10 Americans have had their personal information compromised online. Passwords with commonly used phrases and personal information makes cracking a password drastically easier.</p>
<p>You may have noticed over the years that password requirements have increased in complexity, including recommendations to change your passwords every couple of months. Compiled from industry recommendations, below is a list of passwords requirements you will be asked to test: </p>
<p><strong>Password Requirments:</strong></p>
<ol>
<li>Must be at least 10 characters in length</li>
<li>Must contain at least:<ul>
<li>one lower case letter </li>
<li>one upper case letter </li>
<li>one numeric character </li>
<li>one non-alphanumeric character</li></ul></li>
<li>Must not contain the phrase <code>password</code> (case insensitive)</li>
<li>Must not contain the user's first or last name, e.g., if the user's name is <code>John Smith</code>, then <code>SmItH876!</code> is not a valid password.</li>
</ol>
<p>Here is the dataset that you will investigate this project:</p>
<div style="background-color: #ebf4f7; color: #595959; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/logins.csv</b></div>
Each row represents a login credential. There are no missing values and you can consider the dataset "clean".
<ul>
    <li><b>id:</b> the user's unique ID.</li>
    <li><b>username:</b> the username with the format {firstname}.{lastname}.</li>
    <li><b>password:</b> the password that may or may not meet the requirements. <i>Note, passwords should never be saved in plaintext, always encrypt them when working with real live passwords!</i></li>
</ul>
</div>
<p>Warning: This dataset contains some <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have been filtered, but may still include words that are explicit and offensive.</p>
<p>From here on out, it will be your task to explore and manipulate the existing data until you can answer the two questions described in the instructions panel. Feel free to import as many packages as you need to complete your task, and add cells as necessary. Finally, remember that you are only tested on your answer, not on the methods you use to arrive at the answer!</p>
<p><strong>Note:</strong> To complete this project, you need to know how to manipulate strings in pandas DataFrames and be familiar with regular expressions. Before starting this project we recommend that you have completed the following courses: <a href="https://learn.datacamp.com/courses/data-cleaning-in-python">Data Cleaning in Python</a> and <a href="https://learn.datacamp.com/courses/regular-expressions-in-python">Regular Expressions in Python</a>.</p>

In [161]:
# Use this cell to begin your analysis, and add as many as you would like!
import pandas as pd

In [162]:
logins = pd.read_csv('./datasets/logins.csv')
logins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        982 non-null    int64 
 1   username  982 non-null    object
 2   password  982 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.1+ KB


<ol>
<li>Must be at least 10 characters in length</li>
<li>Must contain at least:<ul>
<li>one lower case letter </li>
<li>one upper case letter </li>
<li>one numeric character </li>
<li>one non-alphanumeric character</li></ul></li>
<li>Must not contain the phrase <code>password</code> (case insensitive)</li>
<li>Must not contain the user's first or last name, e.g., if the user's name is <code>John Smith</code>, then <code>SmItH876!</code> is not a valid password.</li>
</ol>

In [163]:
# Rule 1: Must be at least 10 characters in length
# Mask
len_check = logins['password'].str.len() >= 10
# Valid passwords
temp_valid_pwd = logins[len_check]
# Invalid passwords
invalid_pwd = logins[~len_check]
# Sanity Check
temp_valid_pwd.shape[0], invalid_pwd.shape[0]

(560, 422)

In [164]:
# Rule 2: Must contain at least one lower case letter, one upper case letter, one numeric character and one non-alphanumeric character
# Run check on valid pass
lcase = temp_valid_pwd['password'].str.contains('[a-z]')
ucase = temp_valid_pwd['password'].str.contains('[A-Z]')
ncase = temp_valid_pwd['password'].str.contains('\d')
nacase = temp_valid_pwd['password'].str.contains('\W')
# char check which checks all conditions
char_check = lcase & ncase & ucase & nacase
# Creates dataframe with all condition not present
invalid_pwd = pd.concat([invalid_pwd, temp_valid_pwd[~char_check]], ignore_index=True, axis=0)
# Creates dataframe with condition present
temp_valid_pwd = temp_valid_pwd[char_check]

# Sanity check
temp_valid_pwd.shape[0], invalid_pwd.shape[0]

(258, 724)

In [165]:
# Rule 3: Must not contain phrase password
# Condition checks for phrase
pws_psnt = temp_valid_pwd['password'].str.contains('password', case=False)
# Concatenates data frame with condtion
invalid_pwd = pd.concat([invalid_pwd, temp_valid_pwd[pws_psnt]], ignore_index=True, axis=0)
# Creates dataframe without password phrase
temp_valid_pwd = temp_valid_pwd[~pws_psnt]

# Sanity Check
temp_valid_pwd.shape[0], invalid_pwd.shape[0]

(257, 725)

In [166]:
# # Rule 4: Must not contain the user's firstname or surname and is case insensitive
# # Create a new col in dataframe with first name and last name
temp_valid_pwd[['firstname', 'lastname']] = temp_valid_pwd['username'].str.split('.', expand=True)

for id, row in temp_valid_pwd.iterrows():
    # Checks for rule 4 
    if row['firstname'] in row['password'].lower() or row['lastname'] in row['password'].lower():
        # Drop rows 
        temp_valid_pwd.drop(index=id, inplace=True)
        # Append new data
        invalid_pwd = invalid_pwd.append(row, ignore_index=True)
        
# Sanity Check
temp_valid_pwd.shape[0], invalid_pwd.shape[0]


/tmp/ipykernel_17811/4063634869.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  invalid_pwd = invalid_pwd.append(row, ignore_index=True)
/tmp/ipykernel_17811/4063634869.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  invalid_pwd = invalid_pwd.append(row, ignore_index=True)
/tmp/ipykernel_17811/4063634869.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  invalid_pwd = invalid_pwd.append(row, ignore_index=True)
/tmp/ipykernel_17811/4063634869.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  invalid_pwd = invalid_pwd.append(row, ignore_index=True)
/tmp/ipykernel_17811/4063634869.py:11: FutureWarning: The frame.append metho

(246, 736)

In [167]:
bad_pws = round(invalid_pwd.shape[0] / logins.shape[0], 2)
bad_pws

0.75

In [168]:
email_lst = invalid_pwd['username'].sort_values()
email_lst

405       abdul.rowland
309        addie.cherry
372        adele.moreno
517        adeline.bush
279         adolfo.kane
             ...       
373    yvette.whitfield
232        yvonne.munoz
264        zachary.huff
172        zelma.abbott
49        zelma.rosario
Name: username, Length: 736, dtype: object